In [1]:
import time, os, math, random
from collections import defaultdict
## Return predictive performance of a pathway
import time, os, math, random
from collections import defaultdict
import pandas as pd
import numpy as np
import scipy.stats as stat
from sklearn.preprocessing import StandardScaler # zscore standardization
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import lifelines; from lifelines.statistics import logrank_test; from lifelines import KaplanMeierFitter

## Initialize
#----------------------------------------------------------------------------------------------------------------------

test_types = {'coad':'FLUOROURACIL'}
ML_algorithm_list = ['Ridge', 'SVR', 'LinearRegression']
pathway_source = 'reactome'
n_jobs = 10
network = 'STRING_700'
zscore_cutoff = -1.2816
max_pathway =10
tmp_dir = 'results/single_pathway_predictions'

start_time = time.ctime()


## Parse proximal pathways
#--------------------------------------------------------------------------------------------------------------------
def return_proximal_pathways(network, pathway_source, zscore_cutoff):
	'''
	output = { drug : [ proximal pathways ] }
	'''
	output = defaultdict(list)
	if pathway_source.lower() == 'reactome':
		df = pd.read_csv('../data/coad_blca_organoid_drugs_zscore_result_reactome.txt', sep='\t')
		tmp_drugs = df.columns
		for tmp_drug in tmp_drugs:
			for drug in tmp_drug.split('_'):
				s= df[tmp_drug]
				z=pd.to_numeric(s, errors='coerce')
				pathway_list = list(df.loc[z<= zscore_cutoff, :]['Pathway'])
				output[drug.upper()] = pathway_list
	return output

"""

#Parse Drug IC50: 
#--------------------------------------------------------------------------------------------------------------------
def return_COAD_organoid_drug_response_IC50():
	'''
	output = { sample : { drug : IC50 } }
	output_drugList = [list of drugs tested ]
	---
	median IC50 values from di/triplicates are returned
	'''
	output = {} # { sample : { drug : IC50 } }
	drug_list =set() # [drugs]

	tmp = {} # { sample : { drug : [list of IC50]}}
	f = open('../data/Drug_IC50_67.txt', 'r')
	for line in f:
		line = line.strip().split('\t')
		if 'cell_line' in line[0]:
			sample_list=line[1:]
		else:
			drug, IC50_list = line[0].strip().upper(), list(map(float, line[1:]))
			for index, IC50 in enumerate(IC50_list):
				if IC50=='':
					pass
				else:
					sample = sample_list[index]
					if not sample in output:
						output[sample] = defaultdict(list)
					output[sample][drug.upper()].append(IC50)
			drug_list.add(drug.upper())
	f.close()
	return output, list(drug_list)
"""
## Output file
f=open('%s/single_pathway_predictions.txt'%(tmp_dir), 'w')
s='\t'.join(['cancer_type', 'drug', 'ML', 'pathway_rank', 'pathway', '5yr_responder', '5yr_nonresponder', 'pvalue'])
print(s, file=f)

fr=open('%s/single_pathway_coefficients.txt'%(tmp_dir), 'w')
m='\t'.join(['cancer', 'drug','num_samples','ML', 'pathway', 'reg_coef', 'abs_reg_coef'])
print(m, file=f)

## Analyze
for ML in ML_algorithm_list:
	# set directory
	fo_dir = tmp_dir
	if os.path.isdir('%s/%s'%(fo_dir, ML)) == False:
		os.mkdir('%s/%s'%(fo_dir, ML))
	fo_dir = '%s/%s'%(fo_dir, ML)

	# cancer specific analysis
	for cancer_type in test_types:
		drug = test_types[cancer_type]
		print ('\n\n-----------------\ntesting for %s / %s / %s,  '%(cancer_type, drug, ML), time.ctime())


		## Import data
		c_dir = os.getcwd()
		os.chdir('../utilities')
		with open("Patient_clinical_Data.py") as f:
			code = compile(f.read(), "Patient_clinical_Data.py", 'exec')
			exec(code, globals())
		with open("Preclinical_Model_data.py") as f:
			code = compile(f.read(), "Preclinical_Model_data.py", 'exec')
			exec(code, globals())
		with open("GSEA.py") as f:
			code = compile(f.read(), "GSEA.py", 'exec')
			exec(code, globals())
		with open("parse_Drugbank.py") as f:
			code = compile(f.read(), "parse_Drugbank.py", 'exec')
			exec(code, globals())
		with open("LUAD_Organoid.py") as f:
			code = compile(f.read(), "LUAD_Organoid.py", 'exec')
			exec(code, globals())
            
		nes_dic = parse_ssGSEA_NES(cancer_type, 'organoid', pathway_source) 
		response_dic, drugList = return_COAD_organoid_drug_response_IC50()
		network_dic = return_proximal_pathways(network, pathway_source, zscore_cutoff)# { drug : [ pathways ] }

		drugPat, patDrug = parse_TCGA_drug_treatment_data(cancer_type)
		surDic = parse_TCGA_survival_data_boolean_format(cancer_type)
		patNES = parse_ssGSEA_NES(cancer_type, 'TCGA', pathway_source)
		#print(nes_dic.keys())
		#print(response_dic.keys())  
        
		if pathway_source.lower() == 'reactome':
			feature_list = reactome_genes_uniprot()
			feature_list = feature_list.keys()
		#print(set(feature_list))
		os.chdir(c_dir)

		## feature_list // proximal pathways only
		for sample in nes_dic:
			feature_list = list(set(feature_list) & set(nes_dic[sample].keys()))
		for pat in list(set(surDic.keys())&set(patNES.keys())):
			feature_list = list(set(feature_list)&set(patNES[pat].keys()))
		feature_list = list(set(feature_list) & set(network_dic[drug]))
		print(len(feature_list))
		expList, samples, responses = [], [], []
		for sample in list(set(nes_dic.keys())&set(response_dic.keys())):
			z=list(response_dic[sample].keys())
			for i in range (len(z)):
				if drug == z[i]:
					samples.append(sample)#; samples = sorted(samples, reverse=True)
					responses.append(response_dic[sample][drug])
		for sample in samples:
			tmp = []
			for feature in feature_list:
				if feature in network_dic[drug]: # proximal pathways only
					tmp.append(float(nes_dic[sample][feature]))
			expList.append(tmp)
		scaler = StandardScaler()
		scaler.fit(expList)
		scaled_expList = scaler.transform(expList) # scaled expression
		scaled_expList = np.array(scaled_expList)
        
		# regression (organoid)
		if ML == 'Ridge':
			regr = RidgeCV(cv=3, alphas=np.arange(0.1,1,0.1)).fit(scaled_expList,responses)
		if ML == 'SVR':
			regr = SVR(kernel='linear').fit(scaled_expList,responses)
		if ML == 'LinearRegression':
			regr = LinearRegression().fit(scaled_expList,responses)
		feature_importance = list(regr.coef_)
		if ML == 'SVR':
			feature_importance = feature_importance[0]
 		#print(feature_importance)
		# feature ranks
		coef_dic = {} # { feature : coefficient }
		abs_coef_dic = {}
		for feature, coef in zip(feature_list, feature_importance):
			# print regression coefficients
			pr='\t'.join(map(str, [cancer_type, drug, len(samples), ML, feature, coef, np.abs(coef)]))
			print(pr, file=fr)
			# coefficents
			coef_dic[feature] = coef
			abs_coef_dic[feature] = np.abs(coef)
		#print(abs_coef_dic.values())
		a_set = set()
		a_set.update(abs_coef_dic.values())
		r = {key: rank for rank, key in enumerate(sorted(a_set, reverse=True), 1)}
		feature_rank_dic = {k: r[v] for k,v in abs_coef_dic.items()}
		feature_dict = {value:key for key, value in feature_rank_dic.items()}
		#for value in range(11):
			#print(feature_dict[value+1])
		# scale expressions (patient)
		pat_expDic = {} # { pat : { feature : scaled expression } }
		pat_expList, pat_samples = [], []
		for pat in list(set(surDic.keys())&set(patNES.keys())):
			pat_samples.append(pat)
			tmp = []
			for feature in feature_list:
				tmp.append(patNES[pat][feature])
			pat_expList.append(tmp)
			#print(pat_expList)
		scaler = StandardScaler()
		scaler.fit(pat_expList)
		scaled_pat_expList = scaler.transform(pat_expList)

		for p_index, pat in enumerate(pat_samples):
			pat_expDic[pat] = {}
			for f_index, feature in enumerate(feature_list):
				pat_expDic[pat][feature] = scaled_pat_expList[p_index][f_index]        
		#print(pat_expDic)        
		# single-pathway prediction
		diff_dic={}
		testing_pathway_rank=0
		for path in range(1,max_pathway):
			testing_pathway_rank=path
			features_used=[]
			coef_used=[]
			for fi_index, fi in enumerate(feature_importance):
				feature = feature_list[fi_index]
				if path==feature_rank_dic[feature]:
					features_used=feature; coef_used=fi

					# predicted drug response (patient)
					pred_response = {} # { pat : predicted drug response }
					month_dic, status_dic = defaultdict(list), defaultdict(list)
					fiveYear_dic = {} # { predicted response : 5 year survival }

					for pat in list(set(pat_expDic.keys())&set(drugPat[drug])&set(surDic.keys())):
						#print(pat)
						pred_r = 0
						pred_r += coef * pat_expDic[pat][feature]
						pred_response[pat] = pred_r
					#print(pred_response)
					# classify patients
					response_cutoff = np.median(list(pred_response.values()))
					#print(response_cutoff)
					resic=[]
					resicsam=[]
					nonresic=[]
					nonresicsam=[]
					for pat in pred_response:
						if pred_response[pat] <= response_cutoff:
							cls = 'Responder'
							resic.append(pred_response[pat])
							resicsam.append(pat)
						else:
							cls = 'Nonresponder'
							nonresic.append(pred_response[pat])
							nonresicsam.append(pat)
                
						month_dic[cls].append(surDic[pat]['months'])
						status_dic[cls].append(surDic[pat]['status'])
					#print('Rseponder IC50:') 
					#print(resicsam)
					#for sam,binny,sur in zip(resicsam, resic,month_dic['Responder']):
						#print(sam,binny,sur) 

        
					#print('Nonrseponder IC50:') 
					#print(nonresicsam)
					#for sam1,binny1,sur1 in zip(resicsam, resic,month_dic['Nonresponder']):
						#print(sam1,binny1,sur1)    
					results = logrank_test(month_dic['Responder'], month_dic['Nonresponder'], event_observed_A=status_dic['Responder'], event_observed_B=status_dic['Nonresponder'])
					pvalue = results.p_value
					for cls in month_dic:
						kmf = KaplanMeierFitter()
						kmf.fit(month_dic[cls], status_dic[cls])
						#print(kmf.statistics_)fiveYear_dic[cls] = kmf.predict(
						fiveYear_dic[cls] = kmf.predict(60)
					# draw survival plot
					f = plt.figure(figsize=(4,4))
					ax = f.add_subplot(1,1,1)
					plt.title('%s / %s / %s / %s\npvalue=%.4f\n'%(cancer_type, drug, ML, testing_pathway_rank, pvalue), fontsize=8)
		
					c1 = KaplanMeierFitter()
					ax = c1.fit(month_dic['Responder'], status_dic['Responder'], label='Responder (n=%s)'%len(month_dic['Responder'])).plot(ax=ax, ci_show=True, c='b')
		

					c2 = KaplanMeierFitter()
					ax = c1.fit(month_dic['Nonresponder'], status_dic['Nonresponder'], label='Nonresponder (n=%s)'%len(month_dic['Nonresponder'])).plot(ax=ax, ci_show=True, c='r')
		
					plt.xlabel('Survival (months)')
					plt.ylabel('Percent survival')
					ymin, ymax = 0, 1.1
					plt.ylim(ymin, ymax)
					plt.plot([60, 60], [ymin, ymax], c='k', linestyle='--')
					plt.tight_layout()
					plt.savefig('%s/%s_%s_rank_%s.jpg'%(fo_dir, cancer_type, drug, testing_pathway_rank), format='jpg')
					#plt.savefig('%s/%s_%s_rank_%s.eps'%(fo_dir, cancer_type, drug, testing_pathway_rank), format='eps', dpi=300)
					plt.close()                        
					deep_da=0;lap=0
					res_rin=max(month_dic['Responder'])
					for debo in range(0, int(res_rin)+1):                        
						survival=kmf.predict(debo)
						deep_da=deep_da+(debo*survival)
						lap=lap+debo
					average_res=deep_da/lap
					dada=0;lappy=0
					nres_rin=max(month_dic['Nonresponder'])
					for mono in range(0, int(nres_rin)+1):
						lappy=lappy+mono
						#print('this is 0 th prediction{}'.format(kmf.predict(0)))
						survival2=kmf.predict(mono)
						#print(survival2)
						dada=dada+(mono*survival2)
					a_nonresp=dada/lappy
					#print("Non responder survival probability:",fiveYear_dic['Nonresponder'])
					diff=average_res-a_nonresp
					print(diff)
					diff_dic.update({path:diff})
		show = max(diff_dic, key=diff_dic.get)
		print("optimal pathway no:",show)
		print("optimal pathway is:",feature_dict[show])
		biomarker=feature_dict[show]
		prox=open('../data/coad_blca_organoid_drugs_zscore_result_reactome.txt')
		drugname=[]
		lasi=[]
		for every in prox:
			sbio=every.strip().split('\t')
			bio=sbio[0]
			if bio=='Pathway':
				drugname=sbio[1:]  
			elif bio == biomarker:
				lasi=sbio[1:] 
		ran={key: rank for rank, key in zip(drugname,lasi)}
		less_d=min(ran.keys())
		print(ran[less_d])
print('process complete, start time: %s - end time: %s '%(start_time, time.ctime()))



-----------------
testing for coad / FLUOROURACIL / Ridge,   Thu Apr 28 12:51:05 2022
37
0.022357757556855318
-0.01973486963433646
-0.006938332654036983
-0.008061860983735292
0.01871528079683904
0.012362269559177297
-0.01129838414157347
0.011822406384238482
-0.0025590451120313107
optimal pathway no: 1
optimal pathway is: REACTOME_ACTIVATION_OF_BH3_ONLY_PROTEINS
LENALIDOMIDE


-----------------
testing for coad / FLUOROURACIL / SVR,   Thu Apr 28 12:51:14 2022
37
0.022357757556855318
-0.01973486963433646
-0.006938332654036983
-0.008061860983735292
0.01871528079683904
0.012362269559177297
-0.01129838414157347
0.011822406384238482
0.003077931602962325
optimal pathway no: 1
optimal pathway is: REACTOME_ACTIVATION_OF_BH3_ONLY_PROTEINS
LENALIDOMIDE


-----------------
testing for coad / FLUOROURACIL / LinearRegression,   Thu Apr 28 12:51:22 2022
37
0.022357757556855318
-0.01973486963433646
-0.006938332654036983
-0.008061860983735292
0.01871528079683904
0.012362269559177297
0.003077931602962